# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
source activate pydeck-ee
jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import requests
import ee

## Authentication

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create Map

Next it's time to create a map. Here we create an `ee.Image` object

In [ ]:
# Initialize objects
ee_layers = []
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)

In [4]:
# %%
# Add Earth Engine dataset
geometry = ee.Geometry.Polygon(
        [[[-121.53162002563477, 37.62442917942242],
          [-121.53822898864746, 37.61871860390886],
          [-121.53307914733887, 37.61144378319061],
          [-121.5281867980957, 37.60784010375065],
          [-121.52209281921387, 37.60586820524277],
          [-121.51840209960938, 37.606344185530936],
          [-121.51273727416992, 37.60777210812061],
          [-121.50175094604492, 37.6082480762255],
          [-121.49454116821289, 37.61239566936059],
          [-121.49127960205078, 37.62136999709244],
          [-121.49797439575195, 37.62667249978579],
          [-121.5252685546875, 37.62653654290317]]])

# Load a Landsat 8 image and display the thermal band.
image = ee.Image('LANDSAT/LC8_L1T_TOA/LC80440342014077LGN00').clip(geometry)
view_state = pdk.ViewState(longitude=-121.51385307312012, latitude=37.61767615130697, zoom=14)
#Map.addLayer(image, {'bands': ['B10'], 'min': 270, 'max': 310}, 'LST')
#print(image)

# Threshold the thermal band to find "hot" objects.
hotspots = image.select('B10').gt(303)

# Mask "cold" pixels.
hotspots = hotspots.mask(hotspots)
#Map.addLayer(hotspots, {'palette': 'FF0000'}, 'hotspots')

# Compute the number of pixels in each patch.
patchsize = hotspots.connectedPixelCount(100, False)
ee_layers.append(EarthEngineLayer(ee_object=patchsize, vis_params={}))
largePatches = patchsize.gt(4)
largePatches = largePatches.updateMask(largePatches)
ee_layers.append(EarthEngineLayer(ee_object=largePatches, vis_params={}))

pixelAreaAllPatches = hotspots.multiply(ee.Image.pixelArea())
pixelAreaLargePatch = largePatches.multiply(ee.Image.pixelArea())
areaAllPathces = pixelAreaAllPatches.reduceRegion(**{'reducer':ee.Reducer.sum(),'geometry':geometry})
areaLargePatch = pixelAreaLargePatch.reduceRegion(**{'reducer':ee.Reducer.sum(),'geometry':geometry})

print(areaAllPathces.getInfo())
print(areaLargePatch.getInfo())



Then just pass these layers to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [8]:
r = pdk.Deck(layers=ee_layers, initial_view_state=view_state)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…